# Jupyter Notebook for the Capstone Project of Coursera

In [88]:
#Loading libraries
import pandas as pd
import numpy as np

In [89]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 1: Scrape Wikipedia Page

In [121]:
# Importing libraries
import bs4 as bs
import urllib
import requests
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests 
from pandas.io.json import json_normalize

In [122]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
canada_data = bs.BeautifulSoup(source,'html.parser')

In [123]:
col_nm = ['PostalCode', 'Borough', 'Neighborhood']
city_data = pd.DataFrame(columns = col_nm)

In [124]:
# loop through the data to find postcode, borough and neighborhood
content = canada_data.find('div', class_ = 'mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i += 1
        elif i == 1:
            borough = td.text
            i += 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            
        city_data = city_data.append(
            {'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},
            ignore_index=True
        )

We should now have a data frame

In [125]:
city_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,0,0
1,M1A,Not assigned,0
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M2A,Not assigned,Not assigned


We can clean our data frame

In [126]:
# We remove the entries that don't have a borough (not assigned or 0)
city_data = city_data[city_data.Borough != 'Not assigned']
city_data = city_data[city_data.Borough != 0]

# We reset the index
city_data.reset_index(drop = True, inplace = True)

In [127]:
# If an entry has a borough but no neighborhood, we assign the borough to the neighborhood
i = 0
for i in range(0,city_data.shape[0]):
    if city_data.iloc[i][2] == 'Not assigned':
        city_data.iloc[i][2] = city_data.iloc[i][1]
        i = i+1
        

df = city_data.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Queen's Park,Queen's Park
1,M1B,Scarborough,"Scarborough, Rouge, Rouge, Rouge, Malvern"
2,M1C,Etobicoke,West Deane Park
3,M1C,Scarborough,"West Deane Park, Highland Creek, Highland Cree..."
4,M1E,Etobicoke,Old Burnhamthorpe


We can group the entries

In [128]:
def neighborhood_list(grouped):
    return ', '.join(sorted(grouped['Neighborhood'].to_list()))

grp = df.groupby(['PostalCode', 'Borough'])
df = grp.apply(neighborhood_list).reset_index(name = 'Neighborhood')

In [129]:
print(df.shape)

(191, 3)
